In [1]:
import gc
import numpy as np

"""
화성-서울 네트워크 분석
    집계시간 : 5분
    분석시간 1800~8400
    검지기 : 1099개
"""
import pandas as pd

import os

# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 8400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 램프 전 본선 검지기
before_ramp = [73, 117, 126, 186, 220, 312, 344, 403, 414, 460]

# 램프 후 본선 검지기
after_ramp = [74, 119, 127, 188, 221, 313, 345, 404, 416, 461]

# 유입 검지기
input_ramp = [902, 904, 906, 908, 910]

#유출 검지기
output_ramp = [901, 903, 905, 907, 909]
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 속도 변화율
def speed_mean(original_df):
    copy_df = original_df.copy()

    # 램프 검지기 제외
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도 변화율
def density_mean(speed_df):
    copy_df = speed_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 중차량 혼입률
def heavy_rate(original_df):
    copy_df = original_df.copy()

    # 진입부 검지기 선정
    measurement = enter_line

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 동적 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # 실측용량 C(2차로 4400)
    max_capacity = 4400
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df


# 램프 간섭 영향률
def rfr_rate(original_df):
    copy_df = original_df.copy()
    copy_df["TimeGroup"] = copy_df["TimeGroup"].astype(str)
    main_results=[]
    for i, (before, after) in enumerate(zip(before_ramp, after_ramp)):
        q_before = (copy_df[copy_df["New_Measurement"] == before]
                    .groupby("TimeGroup")
                    .size()
                    .reset_index(name="q_before"))

        q_after = (copy_df[copy_df["New_Measurement"] == after]
                   .groupby("TimeGroup")
                   .size()
                   .reset_index(name="q_after"))

        merged = q_before.merge(q_after, on="TimeGroup", how="outer").fillna(0)
        merged["Qm"] = (merged["q_before"] + merged["q_after"]) / 2
        main_results.append(merged)

    main_df = pd.concat(main_results, ignore_index=True)
    main_df = main_df.groupby("TimeGroup")[["Qm"]].mean().reset_index()


    ramp_results = []
    for input_, output_ in zip(input_ramp, output_ramp):
        q_in = (copy_df[copy_df["New_Measurement"] == input_]
                .groupby("TimeGroup").size().reset_index(name="q_in"))
        q_out = (copy_df[copy_df["New_Measurement"] == output_]
                 .groupby("TimeGroup").size().reset_index(name="q_out"))
        merged = q_in.merge(q_out, on="TimeGroup", how="outer").fillna(0)
        ramp_results.append(merged)

    ramp_df = pd.concat(ramp_results, ignore_index=True)
    ramp_df = ramp_df.groupby("TimeGroup")[["q_in", "q_out"]].mean().reset_index()

    rfr_df = pd.merge(main_df, ramp_df, on="TimeGroup", how="inner")

    # 램프 간섭 영향률 계산
    rfr_df["IR_in"] = rfr_df["q_in"] / rfr_df["Qm"]
    rfr_df["IR_out"] = rfr_df["q_out"] / rfr_df["Qm"]
    rfr_df["RFR"] = rfr_df["IR_in"] + rfr_df["IR_out"]
    return rfr_df

# 진출 원활율
def output_normality(original_df):

    copy_df = original_df.copy()
    copy_df["F(outrate)"] = 0
    display("copy_df : ", copy_df)
    return copy_df


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "rate"]], on=["TimeGroup"])
    .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "F(outrate)"]], on="TimeGroup")
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()

    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    #modify_df = modify_df[(modify_df["StartTime"] >=3300) &(modify_df["StartTime"] < 6900)]
    modify_df = modify_df[(modify_df["StartTime"] >=1800) &(modify_df["StartTime"] < 5400)]
    modify_df = modify_df[~modify_df["New_Measurement"].isin([266, 531])]
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\화성~서울"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)
                display("original_df : ", original_df)

                # 속도변화율
                speed_df = speed_mean(original_df)

                # 밀도변화율
                density_df = density_mean(speed_df)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)
                #display("heavy_df : ", heavy_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)
                #display("entry_saturation_df : ", entry_saturation_df)

                # 램프 간섭 영향률
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(rfr_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)


                delta_v_df = stvm_df[["TimeGroup", "New_Measurement", "delta_V"]].copy()
                delta_pivot_v_df = pivot_table(delta_v_df, "delta_V", preprocess=modify_frame)
                #save_to_excel(delta_pivot_v_df, folder_path, "속도변화값", i)

                delta_k_df = stvm_df[["TimeGroup", "New_Measurement", "delta_K"]].copy()
                delta_pivot_k_df = pivot_table(delta_k_df, "delta_K", preprocess=modify_frame)
                #save_to_excel(delta_pivot_k_df, folder_path, "밀도변화값", i)

                heavy_df = stvm_df[["TimeGroup", "New_Measurement", "rate"]].copy()
                heavy_df = pivot_table(heavy_df, "rate", preprocess=modify_frame)
                #save_to_excel(heavy_df, folder_path, "중차량", i)

                entry_df = stvm_df[["TimeGroup", "New_Measurement", "Phi_진입"]].copy()
                entry_df = pivot_table(entry_df, "Phi_진입", preprocess=modify_frame)
                #save_to_excel(entry_df, folder_path, "진입포화도", i)

                ramp_df = stvm_df[["TimeGroup", "New_Measurement", "RFR"]].copy()
                ramp_df = pivot_table(ramp_df, "RFR", preprocess=modify_frame)
                #save_to_excel(ramp_df, folder_path, "램프유출입", i)


                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                #display("z_score_df : ", z_score_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #display("stvm_pivot_df : ", stvm_pivot_df)
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #display("speed_pivot_df : ", speed_pivot_df)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


'original_df : '

,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],VehLength[m],,New_Measurement,TimeGroup
0,10017,1800.01,-1.0,3135,100,0,95.3,4.76,NaN,17,1800~2100
1,10082,1800.03,-1.0,2727,100,0,98.4,4.64,NaN,82,1800~2100
2,10188,1800.10,-1.0,3188,100,0,77.8,3.75,NaN,188,1800~2100
3,10213,1800.10,-1.0,2567,300,0,78.6,12.40,NaN,213,1800~2100
4,10230,1800.08,-1.0,2924,100,0,98.5,3.75,NaN,230,1800~2100
...,...,...,...,...,...,...,...,...,...,...,...
1957670,20338,8399.98,-1.0,14576,100,0,105.8,4.21,NaN,338,8100~8400
1957671,20458,8399.99,-1.0,13772,300,0,98.7,12.40,NaN,458,8100~8400
1957672,20494,8399.96,-1.0,13819,300,0,99.6,12.40,NaN,494,8100~8400
1957673,20509,8399.96,-1.0,13294,100,0,69.4,4.21,NaN,509,8100~8400


'copy_df : '

,TimeGroup,Qm,q_in,q_out,IR_in,IR_out,RFR,F(outrate)
0,1800~2100,169.45,38.4,37.0,0.226616,0.218353,0.444969,0
1,2100~2400,166.60,40.8,34.4,0.244898,0.206483,0.451381,0
2,2400~2700,175.30,40.2,35.8,0.229321,0.204221,0.433542,0
3,2700~3000,164.75,37.8,37.2,0.229439,0.225797,0.455235,0
4,3000~3300,173.05,40.0,38.2,0.231147,0.220745,0.451893,0
5,3300~3600,170.40,35.8,37.8,0.210094,0.221831,0.431925,0
6,3600~3900,175.85,42.2,38.4,0.239977,0.218368,0.458345,0
7,3900~4200,176.10,36.6,38.6,0.207836,0.219194,0.427030,0
8,4200~4500,171.90,40.6,39.4,0.236184,0.229203,0.465387,0
9,4500~4800,167.00,42.2,37.2,0.252695,0.222754,0.475449,0


'merged_df : '

...

,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],VehLength[m],,New_Measurement,TimeGroup
0,10014,1800.10,-1.0,5367,100,0,67.2,4.61,NaN,14,1800~2100
1,10041,1800.01,-1.0,4980,640,0,79.1,10.21,NaN,41,1800~2100
2,10098,1800.04,-1.0,4041,100,0,39.7,4.64,NaN,98,1800~2100
3,10121,1800.00,-1.0,3149,100,0,71.7,4.21,NaN,121,1800~2100
4,10147,1800.10,-1.0,4451,100,0,81.1,3.75,NaN,147,1800~2100
...,...,...,...,...,...,...,...,...,...,...,...
2588476,20410,8399.98,-1.0,19739,100,0,92.1,4.64,NaN,410,8100~8400
2588477,20431,8399.96,-1.0,17638,100,0,106.0,4.01,NaN,431,8100~8400
2588478,20490,8399.96,-1.0,21205,100,0,100.6,4.64,NaN,490,8100~8400
2588479,20519,8399.94,-1.0,16745,100,0,60.8,4.76,NaN,519,8100~8400


'copy_df : '

,TimeGroup,Qm,q_in,q_out,IR_in,IR_out,RFR,F(outrate)
0,1800~2100,226.45,54.6,50.0,0.241113,0.220799,0.461912,0
1,2100~2400,226.30,57.0,53.0,0.251878,0.234202,0.486080,0
2,2400~2700,223.50,52.2,48.8,0.233557,0.218345,0.451902,0
3,2700~3000,227.55,53.4,50.2,0.234674,0.220611,0.455285,0
4,3000~3300,223.75,51.4,50.2,0.229721,0.224358,0.454078,0
5,3300~3600,221.80,58.0,49.8,0.261497,0.224527,0.486023,0
6,3600~3900,224.65,59.2,55.8,0.263521,0.248386,0.511907,0
7,3900~4200,220.50,59.4,42.4,0.269388,0.192290,0.461678,0
8,4200~4500,223.20,56.0,48.4,0.250896,0.216846,0.467742,0
9,4500~4800,224.30,56.6,49.6,0.252341,0.221132,0.473473,0


'merged_df : '

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,F(outrate)
0,1800~2100,1,0.093617,-0.071622,0.258530,0.891818,0.461912,0
1,1800~2100,2,0.224113,-0.188003,0.258530,0.905455,0.461912,0
2,1800~2100,3,0.143622,-0.173281,0.258530,0.900000,0.461912,0
3,1800~2100,4,0.065101,-0.127324,0.258530,0.850909,0.461912,0
4,1800~2100,5,0.045897,-0.099932,0.258530,0.790909,0.461912,0
...,...,...,...,...,...,...,...,...
11611,8100~8400,527,0.020283,-0.052120,0.246113,0.829091,0.444739,0
11612,8100~8400,528,-0.006100,-0.014396,0.246113,0.801818,0.444739,0
11613,8100~8400,529,-0.017531,-0.017498,0.246113,0.785455,0.444739,0
11614,8100~8400,530,0.044865,-0.060152,0.246113,0.758182,0.444739,0
